In [1]:
import os
print("Current Directory:", os.getcwd())

%cd F:\FELIPE\Trabajos extras\2024\RHETT MORRIS\Car accident fatatilities
print("Changed to Directory:", os.getcwd())


Current Directory: C:\Users\Felipe
F:\FELIPE\Trabajos extras\2024\RHETT MORRIS\Car accident fatatilities
Changed to Directory: F:\FELIPE\Trabajos extras\2024\RHETT MORRIS\Car accident fatatilities


In [2]:
import pandas as pd
import numpy as np

accident_data1222 = pd.read_excel('F:/FELIPE/Trabajos extras/2024/RHETT MORRIS/Car accident fatatilities/2012-2022.xlsx', index_col=0)


In [3]:
import geopandas as gpd

In [4]:
df_zcta = gpd.read_file('F:/FELIPE/Trabajos extras/2024/RHETT MORRIS/Car accident fatatilities/tl_2022_us_zcta520.zip')

In [5]:
gdf_accident_data1222 = gpd.GeoDataFrame(accident_data1222, geometry=gpd.points_from_xy(x=accident_data1222["LONGITUD"], y=accident_data1222["LATITUDE"]), crs="epsg:4269")

In [6]:
gdf_accident_data1222 = gpd.sjoin(gdf_accident_data1222.to_crs(epsg=4269), df_zcta, how="left", predicate="intersects")

In [7]:
gdf_accident_data1222['ZCTA5CE20'] = pd.to_numeric(gdf_accident_data1222['ZCTA5CE20'])

In [8]:
population = pd.read_csv('F:/FELIPE/Trabajos extras/2024/RHETT MORRIS/Car accident fatatilities/DECENNIALDP2020.DP1-Data.csv')


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\3368768289.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161) have mixed types. Specify dtype option on import or set low_memory=False.
  population = pd.read_csv('F:/FELIPE/Trabajos extras/2024/RHETT MORRIS/Car accident fatatilities/DECENNIALDP2020.DP1-Data.csv')


In [9]:
population['NAME'] = population['NAME'].str.replace('ZCTA5 ', '')

In [10]:
population = population.rename(columns={'NAME': 'ZCTA5CE20'})
population = population.drop(0)

In [11]:
population['ZCTA5CE20'] = pd.to_numeric(population['ZCTA5CE20'])
population['DP1_0001C'] = pd.to_numeric(population['DP1_0001C'])

In [12]:
merged_df = pd.merge(gdf_accident_data1222, population, on='ZCTA5CE20', how='left')

In [13]:
#Table A: Value = the number of incidents with any motor vehicle fatality in each year

unique_years = merged_df['YEAR'].unique()


for year in unique_years:
    merged_df[f'Count_{year}'] = merged_df.apply(lambda col: col['FATALS'] if col['YEAR'] == year else None, axis=1)



In [14]:
merged_df = merged_df.rename(columns={'Count_2022.0': '2022',  'Count_2021.0': '2021', 'Count_2020.0': '2020','Count_2019.0': '2019',  'Count_2018.0': '2018', 'Count_2017.0': '2017','Count_2016.0': '2016',  'Count_2015.0': '2015', 'Count_2014.0': '2014','Count_2013.0': '2013',  'Count_2012.0': '2012'})

In [15]:

# Group by 'ZCTACE20' and count cases
grouped_df = merged_df.groupby('ZCTA5CE20').agg({
    '2012': 'count',
    '2013': 'count',
    '2014': 'count',
    '2015': 'count',
    '2016': 'count',
    '2017': 'count',
    '2018': 'count',
    '2019': 'count',
    '2020': 'count',
    '2021': 'count',
    '2022': 'count',
    'FATALS': 'count',    
    'DP1_0001C':'mean'
}).reset_index()

In [16]:
us_info = pd.read_csv('F:/FELIPE/Trabajos extras/2024/RHETT MORRIS/Car accident fatatilities/uszips.csv')
us_info = us_info.rename(columns={'zip': 'ZCTA5CE20'})

In [17]:
grouped_df = pd.merge(grouped_df, us_info, on='ZCTA5CE20', how='left')

In [18]:
grouped_df = grouped_df[['ZCTA5CE20','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','state_name','county_name','DP1_0001C']]
grouped_df = grouped_df.rename(columns={'state_name': 'STATE',  'county_name': 'COUNTY', 'DP1_0001C': 'POPULATION'})

In [19]:
grouped_df.to_excel("C:/Users/Felipe/Documents/totalincidents.xlsx", sheet_name='Sheet1', index=False)

In [20]:
merged_df_pedestrian = pd.merge(gdf_accident_data1222, population, on='ZCTA5CE20', how='left')

In [21]:
merged_df_pedestrian = merged_df_pedestrian[merged_df_pedestrian['COUNT PEDERESTRIANS AND IN/ON BUILDING FATALITIES'] > 0]

In [22]:
#Table B: Value = the number of incidents with a PEDESTRIAN fatality in each year

unique_years = merged_df_pedestrian['YEAR'].unique()



for year in unique_years:
    merged_df_pedestrian[f'Count_{year}'] = merged_df_pedestrian.apply(lambda col: col['COUNT PEDERESTRIANS AND IN/ON BUILDING FATALITIES'] if col['YEAR'] == year else None, axis=1)

merged_df_pedestrian.head()

,YEAR,MONTH,MONTHNAME,DAY,DAYNAME,DAY_WEEK,DAY_WEEKNAME,HOUR,MINUTE,LATITUDE,...,Count_2021.0,Count_2020.0,Count_2019.0,Count_2018.0,Count_2017.0,Count_2016.0,Count_2015.0,Count_2014.0,Count_2013.0,Count_2012.0
4,2022.0,1,January,2,2,1.0,Sunday,18,48,33.540281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022.0,1,January,2,2,1.0,Sunday,18,28,32.351689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2022.0,1,January,9,9,1.0,Sunday,0,30,30.464969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2022.0,1,January,15,15,7.0,Saturday,23,9,33.590283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2022.0,1,January,16,16,1.0,Sunday,23,0,33.665125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
merged_df_pedestrian = merged_df_pedestrian.rename(columns={'Count_2022.0': '2022',  'Count_2021.0': '2021', 'Count_2020.0': '2020','Count_2019.0': '2019',  'Count_2018.0': '2018', 'Count_2017.0': '2017','Count_2016.0': '2016',  'Count_2015.0': '2015', 'Count_2014.0': '2014','Count_2013.0': '2013',  'Count_2012.0': '2012'})

In [24]:

# Group by 'ZCTACE20' and count cases
grouped_df_pedestrian = merged_df_pedestrian.groupby('ZCTA5CE20').agg({
    '2012': 'count',
    '2013': 'count',
    '2014': 'count',
    '2015': 'count',
    '2016': 'count',
    '2017': 'count',
    '2018': 'count',
    '2019': 'count',
    '2020': 'count',
    '2021': 'count',
    '2022': 'count',
    'COUNT PEDERESTRIANS AND IN/ON BUILDING FATALITIES': 'count',    
    'DP1_0001C':'mean'
}).reset_index()

print(grouped_df_pedestrian)


       ZCTA5CE20  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  \
0         1001.0     0     0     0     0     0     1     0     0     0     0   
1         1002.0     0     0     1     0     1     0     0     2     0     0   
2         1003.0     0     0     0     0     0     0     0     0     0     0   
3         1005.0     0     0     0     0     1     0     0     0     0     0   
4         1007.0     0     0     0     1     0     1     0     0     0     0   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
13806    99749.0     0     0     1     0     0     0     0     0     0     0   
13807    99752.0     0     0     0     0     0     1     0     0     0     1   
13808    99762.0     0     0     1     0     0     0     0     0     0     0   
13809    99801.0     0     0     0     0     1     0     2     0     0     1   
13810    99901.0     0     0     0     0     0     1     0     0     1     0   

       2022  COUNT PEDERESTRIANS AND IN

In [25]:
grouped_df_pedestrian = pd.merge(grouped_df_pedestrian, us_info, on='ZCTA5CE20', how='left')

In [26]:
grouped_df_pedestrian = grouped_df_pedestrian[['ZCTA5CE20','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','state_name','county_name','DP1_0001C']]
grouped_df_pedestrian = grouped_df_pedestrian.rename(columns={'state_name': 'STATE',  'county_name': 'COUNTY', 'DP1_0001C': 'POPULATION'})

In [27]:
grouped_df_pedestrian.to_excel("C:/Users/Felipe/Documents/pedestrian.xlsx", sheet_name='Sheet1', index=False)

In [28]:
merged_df_cyclist = pd.merge(gdf_accident_data1222, population, on='ZCTA5CE20', how='left')

In [29]:
merged_df_cyclist = merged_df_cyclist[merged_df_cyclist['COUNT BICYCLIST, OTHER PEDALCYCLIST AND PERSONAL CONVEYANCE FATALITIES'] > 0]

In [30]:

#Table C: Value = the number of incidents with a CYCLIST fatality in each year

unique_years = merged_df_cyclist['YEAR'].unique()


for year in unique_years:
    merged_df_cyclist[f'Count_{year}'] = merged_df_cyclist.apply(lambda row: row['COUNT BICYCLIST, OTHER PEDALCYCLIST AND PERSONAL CONVEYANCE FATALITIES'] if row['YEAR'] == year else None, axis=1)


In [31]:
merged_df_cyclist = merged_df_cyclist.rename(columns={'Count_2022.0': '2022',  'Count_2021.0': '2021', 'Count_2020.0': '2020','Count_2019.0': '2019',  'Count_2018.0': '2018', 'Count_2017.0': '2017','Count_2016.0': '2016',  'Count_2015.0': '2015', 'Count_2014.0': '2014','Count_2013.0': '2013',  'Count_2012.0': '2012'})

In [32]:

# Group by 'ZCTACE20' and count cases
grouped_df_cyclist = merged_df_cyclist.groupby('ZCTA5CE20').agg({
    '2012': 'count',
    '2013': 'count',
    '2014': 'count',
    '2015': 'count',
    '2016': 'count',
    '2017': 'count',
    '2018': 'count',
    '2019': 'count',
    '2020': 'count',
    '2021': 'count',
    '2022': 'count',
    'COUNT BICYCLIST, OTHER PEDALCYCLIST AND PERSONAL CONVEYANCE FATALITIES': 'count',
    'DP1_0001C':'mean'
}).reset_index()

print(grouped_df_cyclist)




      ZCTA5CE20  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  \
0        1002.0     0     1     0     0     0     0     0     0     0     0   
1        1013.0     0     0     0     0     0     0     0     0     0     0   
2        1028.0     0     0     0     0     0     0     0     0     0     0   
3        1040.0     0     0     0     0     1     0     0     0     0     0   
4        1060.0     1     0     0     0     0     0     0     0     0     1   
...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
5388    99671.0     0     0     0     0     0     0     0     0     1     0   
5389    99705.0     1     0     0     0     0     0     0     0     0     0   
5390    99730.0     0     0     0     0     0     0     0     0     1     0   
5391    99743.0     0     0     1     0     0     0     0     0     0     0   
5392    99835.0     0     0     0     0     0     0     0     0     0     1   

      2022  \
0        0   
1        1   
2        

In [33]:
grouped_df_cyclist = pd.merge(grouped_df_cyclist, us_info, on='ZCTA5CE20', how='left')

In [34]:
grouped_df_cyclist = grouped_df_cyclist[['ZCTA5CE20','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','state_name','county_name','DP1_0001C']]
grouped_df_cyclist = grouped_df_cyclist.rename(columns={'state_name': 'STATE',  'county_name': 'COUNTY', 'DP1_0001C': 'POPULATION'})

In [35]:
grouped_df_cyclist.to_excel("C:/Users/Felipe/Documents/cyclists.xlsx", sheet_name='Sheet1', index=False)

In [36]:
merged_df_driverandpassanger = pd.merge(gdf_accident_data1222, population, on='ZCTA5CE20', how='left')

In [37]:
merged_df_driverandpassanger['TOTAL FATALITIES'] = merged_df_driverandpassanger['COUNT DRIVER FATALITIES'] + merged_df_driverandpassanger['COUNT PASSENGER AND OCCUPANTS FATALITIES']

In [38]:
merged_df_driverandpassanger = merged_df_driverandpassanger[merged_df_driverandpassanger['TOTAL FATALITIES'] > 0]

In [39]:

#Table D: Value = the number of incidents with a DRIVER OR PASSENGER fatality in each year

unique_years = merged_df_driverandpassanger['YEAR'].unique()


for year in unique_years:
    merged_df_driverandpassanger[f'Count_{year}'] = merged_df_driverandpassanger.apply(lambda col: col['TOTAL FATALITIES'] if col['YEAR'] == year else None, axis=1)



In [40]:
merged_df_driverandpassanger = merged_df_driverandpassanger.rename(columns={'Count_2022.0': '2022',  'Count_2021.0': '2021', 'Count_2020.0': '2020','Count_2019.0': '2019',  'Count_2018.0': '2018', 'Count_2017.0': '2017','Count_2016.0': '2016',  'Count_2015.0': '2015', 'Count_2014.0': '2014','Count_2013.0': '2013',  'Count_2012.0': '2012'})

In [41]:

# Group by 'ZCTACE20' and count cases
grouped_df_driverandpassanger = merged_df_driverandpassanger.groupby('ZCTA5CE20').agg({
    '2012': 'count',
    '2013': 'count',
    '2014': 'count',
    '2015': 'count',
    '2016': 'count',
    '2017': 'count',
    '2018': 'count',
    '2019': 'count',
    '2020': 'count',
    '2021': 'count',
    '2022': 'count',
    'TOTAL FATALITIES': 'count',
    'DP1_0001C':'mean'
}).reset_index()

print(grouped_df_driverandpassanger)


       ZCTA5CE20  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  \
0         1001.0     0     1     1     0     1     1     0     0     1     0   
1         1002.0     0     0     1     0     1     1     0     0     1     0   
2         1005.0     0     0     1     0     1     1     1     0     0     2   
3         1007.0     2     0     1     0     0     1     2     1     0     2   
4         1008.0     0     0     0     0     2     0     0     1     0     0   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
27206    99901.0     1     0     0     0     1     0     1     0     0     0   
27207    99921.0     0     1     0     1     0     1     1     0     0     0   
27208    99923.0     0     0     0     0     0     1     0     0     0     0   
27209    99925.0     0     0     0     0     1     0     0     0     0     1   
27210    99929.0     0     0     0     0     0     0     2     0     0     0   

       2022  TOTAL FATALITIES  DP1_0001

In [42]:
grouped_df_driverandpassanger = pd.merge(grouped_df_driverandpassanger, us_info, on='ZCTA5CE20', how='left')

In [43]:
grouped_df_driverandpassanger = grouped_df_driverandpassanger[['ZCTA5CE20','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','state_name','county_name','DP1_0001C']]
grouped_df_driverandpassanger = grouped_df_driverandpassanger.rename(columns={'state_name': 'STATE',  'county_name': 'COUNTY', 'DP1_0001C': 'POPULATION'})

In [44]:
grouped_df_driverandpassanger.to_excel("C:/Users/Felipe/Documents/driverandpassanger.xlsx", sheet_name='Sheet1', index=False)

In [45]:
#Table E: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with any motor vehicle fatality in each year
grouped_df_Inc100000 = grouped_df.copy()

In [46]:
#Table E: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with any motor vehicle fatality in each year
grouped_df_Inc100000['2012'] = (grouped_df_Inc100000['2012']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2013'] = (grouped_df_Inc100000['2013']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2014'] = (grouped_df_Inc100000['2014']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2015'] = (grouped_df_Inc100000['2015']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2016'] = (grouped_df_Inc100000['2016']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2017'] = (grouped_df_Inc100000['2017']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2018'] = (grouped_df_Inc100000['2018']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2019'] = (grouped_df_Inc100000['2019']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2020'] = (grouped_df_Inc100000['2020']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2021'] = (grouped_df_Inc100000['2021']/grouped_df_Inc100000['POPULATION'])*100000
grouped_df_Inc100000['2022'] = (grouped_df_Inc100000['2022']/grouped_df_Inc100000['POPULATION'])*100000


In [47]:
grouped_df_Inc100000.to_excel("C:/Users/Felipe/Documents/totalx100.xlsx", sheet_name='Sheet1', index=False)

In [48]:
#Table F: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with a PEDESTRIAN fatality in each year
grouped_df_pedestrian_Inc100000 = grouped_df_pedestrian.copy()

In [49]:
#Table F: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with a PEDESTRIAN fatality in each year



grouped_df_pedestrian_Inc100000['2012'] = (grouped_df_pedestrian_Inc100000['2012']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2013'] = (grouped_df_pedestrian_Inc100000['2013']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2014'] = (grouped_df_pedestrian_Inc100000['2014']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2015'] = (grouped_df_pedestrian_Inc100000['2015']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2016'] = (grouped_df_pedestrian_Inc100000['2016']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2017'] = (grouped_df_pedestrian_Inc100000['2017']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2018'] = (grouped_df_pedestrian_Inc100000['2018']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2019'] = (grouped_df_pedestrian_Inc100000['2019']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2020'] = (grouped_df_pedestrian_Inc100000['2020']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2021'] = (grouped_df_pedestrian_Inc100000['2021']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000
grouped_df_pedestrian_Inc100000['2022'] = (grouped_df_pedestrian_Inc100000['2022']/grouped_df_pedestrian_Inc100000['POPULATION'])*100000


In [50]:
grouped_df_pedestrian_Inc100000.to_excel("C:/Users/Felipe/Documents/pedrestianx100.xlsx", sheet_name='Sheet1', index=False)

In [51]:
grouped_df_cyclist_Inc100000 = grouped_df_cyclist.copy()

In [52]:
#Table G: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with a CYCLIST fatality in each year


grouped_df_cyclist_Inc100000['2012'] = (grouped_df_cyclist_Inc100000['2012']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2013'] = (grouped_df_cyclist_Inc100000['2013']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2014'] = (grouped_df_cyclist_Inc100000['2014']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2015'] = (grouped_df_cyclist_Inc100000['2015']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2016'] = (grouped_df_cyclist_Inc100000['2016']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2017'] = (grouped_df_cyclist_Inc100000['2017']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2018'] = (grouped_df_cyclist_Inc100000['2018']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2019'] = (grouped_df_cyclist_Inc100000['2019']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2020'] = (grouped_df_cyclist_Inc100000['2020']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2021'] = (grouped_df_cyclist_Inc100000['2021']/grouped_df_cyclist_Inc100000['POPULATION'])*100000
grouped_df_cyclist_Inc100000['2022'] = (grouped_df_cyclist_Inc100000['2022']/grouped_df_cyclist_Inc100000['POPULATION'])*100000



In [53]:
grouped_df_cyclist_Inc100000.to_excel("C:/Users/Felipe/Documents/cyclistx100.xlsx", sheet_name='Sheet1', index=False)

In [54]:
grouped_df_driverandpassanger_Inc100000 = grouped_df_driverandpassanger.copy()

In [55]:
# Table H: Value = the number of incidents PER 100,000 RESIDENTS IN THE POPULATION with a DRIVER OR PASSENGER fatality in each year


grouped_df_driverandpassanger_Inc100000['2012'] = (grouped_df_driverandpassanger_Inc100000['2012']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2013'] = (grouped_df_driverandpassanger_Inc100000['2013']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2014'] = (grouped_df_driverandpassanger_Inc100000['2014']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2015'] = (grouped_df_driverandpassanger_Inc100000['2015']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2016'] = (grouped_df_driverandpassanger_Inc100000['2016']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2017'] = (grouped_df_driverandpassanger_Inc100000['2017']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2018'] = (grouped_df_driverandpassanger_Inc100000['2018']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2019'] = (grouped_df_driverandpassanger_Inc100000['2019']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2020'] = (grouped_df_driverandpassanger_Inc100000['2020']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2021'] = (grouped_df_driverandpassanger_Inc100000['2021']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000
grouped_df_driverandpassanger_Inc100000['2022'] = (grouped_df_driverandpassanger_Inc100000['2022']/grouped_df_driverandpassanger_Inc100000['POPULATION'])*100000


In [56]:
grouped_df_driverandpassanger_Inc100000.to_excel("C:/Users/Felipe/Documents/driverandpassangerx100.xlsx", sheet_name='Sheet1', index=False)

In [58]:
incidentsgreater1000 = grouped_df[grouped_df['POPULATION'] >= 1000]
incidentsgreater1000['2012-2022']  = incidentsgreater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)

max_values_idx = incidentsgreater1000['2012-2022'].idxmax()
max_values_ZCTA5CE20 = incidentsgreater1000.loc[max_values_idx]

print("\nDataFrame with maximum values:")
print(max_values_ZCTA5CE20)


DataFrame with maximum values:
ZCTA5CE20        93307.0
2012                  17
2013                  15
2014                  11
2015                  17
2016                  18
2017                  20
2018                  13
2019                  13
2020                  28
2021                  24
2022                  22
STATE         California
COUNTY              Kern
POPULATION       88338.0
2012-2022            198
Name: 25641, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\1795954929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidentsgreater1000['2012-2022']  = incidentsgreater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [59]:
incidentsgreater1000_100k = grouped_df_Inc100000[grouped_df_Inc100000['POPULATION'] >= 1000]

incidentsgreater1000_100k['2012-2022']  = incidentsgreater1000_100k[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)

max_val_100k_idx = incidentsgreater1000_100k['2012-2022'].idxmax()
max_val_100k_ZCTA5CE20 = incidentsgreater1000_100k.loc[max_val_100k_idx]

print("\nDataFrame with maximum values:")
print(max_val_100k_ZCTA5CE20)


DataFrame with maximum values:
ZCTA5CE20         93243.0
2012           432.900433
2013           144.300144
2014            72.150072
2015           288.600289
2016           505.050505
2017           288.600289
2018           865.800866
2019           432.900433
2020           505.050505
2021           721.500722
2022           505.050505
STATE          California
COUNTY               Kern
POPULATION         1386.0
2012-2022     4761.904762
Name: 25606, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\414031202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidentsgreater1000_100k['2012-2022']  = incidentsgreater1000_100k[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [60]:
inc_pedestrian_greater1000 = grouped_df_pedestrian[grouped_df_pedestrian['POPULATION'] >= 1000]
inc_pedestrian_greater1000['2012-2022']  = inc_pedestrian_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)

max_val_ped_idx = inc_pedestrian_greater1000['2012-2022'].idxmax()
max_val_ped_ZCTA5CE20 = inc_pedestrian_greater1000.loc[max_val_ped_idx]

print("\nDataFrame with maximum values:")
print(max_val_ped_ZCTA5CE20)




DataFrame with maximum values:
ZCTA5CE20         90044.0
2012                    3
2013                    0
2014                    3
2015                    5
2016                   10
2017                    9
2018                    8
2019                   13
2020                    6
2021                    7
2022                    8
STATE          California
COUNTY        Los Angeles
POPULATION        94354.0
2012-2022              72
Name: 12086, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\370432549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_pedestrian_greater1000['2012-2022']  = inc_pedestrian_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [61]:
inc_ped_100K_greater1000 = grouped_df_pedestrian_Inc100000[grouped_df_pedestrian_Inc100000['POPULATION'] >= 1000]
inc_ped_100K_greater1000 ['2012-2022']  = inc_ped_100K_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)



max_val_ped_100K_idx = inc_ped_100K_greater1000['2012-2022'].idxmax()
max_val_ped_100K_ZCTA5CE20 = inc_ped_100K_greater1000.loc[max_val_ped_100K_idx]

print("\nDataFrame with maximum values:")
print(max_val_ped_100K_ZCTA5CE20)




DataFrame with maximum values:
ZCTA5CE20         30336.0
2012            74.738416
2013                  0.0
2014                  0.0
2015           149.476831
2016                  0.0
2017           373.692078
2018           224.215247
2019           298.953662
2020           298.953662
2021           298.953662
2022           149.476831
STATE             Georgia
COUNTY             Fulton
POPULATION         1338.0
2012-2022     1868.460389
Name: 4089, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\875461062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_ped_100K_greater1000 ['2012-2022']  = inc_ped_100K_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [62]:
inc_cyclist_greater1000 = grouped_df_cyclist[grouped_df_cyclist['POPULATION'] >= 1000]
inc_cyclist_greater1000['2012-2022'] = inc_cyclist_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)

max_val_cyc_idx = inc_cyclist_greater1000['2012-2022'].idxmax()
max_val_cyc_ZCTA5CE20 = inc_cyclist_greater1000.loc[max_val_cyc_idx]

print("\nDataFrame with maximum values:")
print(max_val_cyc_ZCTA5CE20)



DataFrame with maximum values:
ZCTA5CE20     34667.0
2012                1
2013                0
2014                0
2015                0
2016                0
2017                1
2018                2
2019                0
2020                4
2021                4
2022                1
STATE         Florida
COUNTY          Pasco
POPULATION    34910.0
2012-2022          13
Name: 1968, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\4071820995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_cyclist_greater1000['2012-2022'] = inc_cyclist_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [63]:
Second_Maximum_Cyclst_cases = inc_cyclist_greater1000.loc[4318]

print("\nDataFrame with second maximum values:")
print(Second_Maximum_Cyclst_cases)


DataFrame with second maximum values:
ZCTA5CE20     85705.0
2012                0
2013                0
2014                2
2015                1
2016                0
2017                2
2018                1
2019                2
2020                2
2021                2
2022                1
STATE         Arizona
COUNTY           Pima
POPULATION    56711.0
2012-2022          13
Name: 4318, dtype: object


In [64]:
inc_cyclist_100k_greater1000 = grouped_df_cyclist_Inc100000[grouped_df_cyclist_Inc100000['POPULATION'] >= 1000]
inc_cyclist_100k_greater1000['2012-2022'] = inc_cyclist_100k_greater1000[['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)

max_val_cyc_100_kidx = inc_cyclist_100k_greater1000['2012-2022'].idxmax()
max_val_cyc_ZCTA5CE20 = inc_cyclist_100k_greater1000.loc[max_val_cyc_100_kidx]

print("\nDataFrame with maximum values:")
print(max_val_cyc_ZCTA5CE20)



DataFrame with maximum values:
ZCTA5CE20        98134.0
2012                 0.0
2013           82.236842
2014                 0.0
2015                 0.0
2016                 0.0
2017          164.473684
2018                 0.0
2019           82.236842
2020                 0.0
2021                 0.0
2022          246.710526
STATE         Washington
COUNTY              King
POPULATION        1216.0
2012-2022     575.657895
Name: 5307, dtype: object


C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\377668926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_cyclist_100k_greater1000['2012-2022'] = inc_cyclist_100k_greater1000 [['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']].sum(axis=1)


In [65]:
incidentsgreater1000['AVG1319'] = incidentsgreater1000[['2013', '2014', '2015','2016','2017','2018','2019']].mean(axis=1)
inc_pedestrian_greater1000['AVG1319'] = inc_pedestrian_greater1000[['2013', '2014', '2015','2016','2017','2018','2019']].mean(axis=1)
inc_cyclist_greater1000['AVG1319'] = inc_cyclist_greater1000[['2013', '2014', '2015','2016','2017','2018','2019']].mean(axis=1)

incidentsgreater1000['AVG2022'] = incidentsgreater1000[['2020', '2021', '2022']].mean(axis=1)
inc_pedestrian_greater1000['AVG2022'] = inc_pedestrian_greater1000[['2020', '2021', '2022']].mean(axis=1)
inc_cyclist_greater1000['AVG2022'] = inc_cyclist_greater1000[['2020', '2021', '2022']].mean(axis=1)

incidentsgreater1000['%_Change'] =  ((incidentsgreater1000['AVG2022']-incidentsgreater1000['AVG1319'])/incidentsgreater1000['AVG1319'])*100
inc_pedestrian_greater1000['%_Change'] = ((inc_pedestrian_greater1000['AVG2022']-inc_pedestrian_greater1000['AVG1319'])/inc_pedestrian_greater1000['AVG1319'])*100
inc_cyclist_greater1000['%_Change']   = ((inc_cyclist_greater1000['AVG2022'] -inc_cyclist_greater1000['AVG1319'])/inc_cyclist_greater1000['AVG1319'])*100

C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\1365557626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidentsgreater1000['AVG1319'] = incidentsgreater1000[['2013', '2014', '2015','2016','2017','2018','2019']].mean(axis=1)
C:\Users\Felipe\AppData\Local\Temp\ipykernel_27452\1365557626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc_pedestrian_greater1000['AVG1319'] = inc_pedestrian_greater1000[['2013', '2014', '2015','2016','2017','2018','2019']].mean(axis=1)
C:\Users\Felipe\AppData\Loca

In [66]:
inc_cleaned = incidentsgreater1000[~incidentsgreater1000['%_Change'].isin([np.inf])]

max_change_idx = inc_cleaned['%_Change'].idxmax()
max_change_ZCTA5CE20 = inc_cleaned.loc[max_change_idx]

print("\ZCTA5CE20 with maximum change:")
print(max_change_ZCTA5CE20)


\ZCTA5CE20 with maximum change:
ZCTA5CE20         32601.0
2012                    1
2013                    0
2014                    0
2015                    0
2016                    0
2017                    1
2018                    0
2019                    0
2020                    4
2021                    0
2022                    6
STATE             Florida
COUNTY            Alachua
POPULATION        22039.0
2012-2022              12
AVG1319          0.142857
AVG2022          3.333333
%_Change      2233.333333
Name: 8427, dtype: object


In [67]:
inc_ped_cleaned = inc_pedestrian_greater1000[~inc_pedestrian_greater1000['%_Change'].isin([np.inf])]

max_change_ped_idx = inc_ped_cleaned['%_Change'].idxmax()
max_change_ped_ZCTA5CE20 = inc_ped_cleaned.loc[max_change_ped_idx]

print("\ZCTA5CE20 with maximum change:")
print(max_change_ped_ZCTA5CE20)


\ZCTA5CE20 with maximum change:
ZCTA5CE20         34474.0
2012                    1
2013                    0
2014                    1
2015                    0
2016                    0
2017                    0
2018                    0
2019                    0
2020                    5
2021                    1
2022                    2
STATE             Florida
COUNTY             Marion
POPULATION        17781.0
2012-2022              10
AVG1319          0.142857
AVG2022          2.666667
%_Change      1766.666667
Name: 5103, dtype: object


In [68]:
inc_cyc_cleaned = inc_cyclist_greater1000[~inc_cyclist_greater1000['%_Change'].isin([np.inf])]

max_change_cyc_idx = inc_cyc_cleaned['%_Change'].idxmax()
max_change_cyc_ZCTA5CE20 = inc_cyc_cleaned.loc[max_change_cyc_idx]

print("\ZCTA5CE20 with maximum change:")
print(max_change_cyc_ZCTA5CE20)

\ZCTA5CE20 with maximum change:
ZCTA5CE20         70116.0
2012                    0
2013                    0
2014                    0
2015                    0
2016                    1
2017                    0
2018                    0
2019                    0
2020                    2
2021                    2
2022                    1
STATE           Louisiana
COUNTY            Orleans
POPULATION        10676.0
2012-2022               6
AVG1319          0.142857
AVG2022          1.666667
%_Change      1066.666667
Name: 3408, dtype: object
